# Workflow

In [227]:
import pandas as pd
import optuna
import ast
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix, matthews_corrcoef, recall_score, precision_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier

data_dir = '/home/darshana/Projects/druggable_proteins/processed_dataset'
feature_engineered_data_dir = '/home/darshana/Projects/druggable_proteins/feature_engineered_dataset'

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)


def evaluate_model(name, model, X_train, y_train, X_test, y_test, results_dataframe, feature_type):
    # evaluate model
    scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
    accuracy = scores.mean()

    # fit the model on the training set
    model.fit(X_train, y_train)

    # predict the test set results
    y_pred = model.predict(X_test)

    # compute the confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # calculate precision, recall (sensitivity), f1-score
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # calculate specificity
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn+fp)

    # calculate MCC
    mcc = matthews_corrcoef(y_test, y_pred)

    temp_df = pd.DataFrame({
        'feature_type': feature_type, 
        'model': name, 
        'with_hypertuning': False,
        'best_params': 'None',
        'accuracy': accuracy, 
        'sensitivity': recall, 
        'specificity': specificity, 
        'precision': precision, 
        'f1': f1, 
        'mcc': mcc,
        'index': f'{feature_type}_{name}_no_hypertuning'
        }, index=['index'])
    # results_dataframe is an empty dataframe to store results with the columns feature_type, model, with_hypertuning, accuracy, sensitivity, specificity, precision, f1, mcc
    return pd.concat([results_dataframe, temp_df])


def optimize_hyperparameters(name, model, objective, trials, results_dataframe, feature_type, X_train, y_train, X_test, y_test):
    def optuna_objective(trial):
        params = objective(trial)
        model_instance = model(**params)
        model_instance.fit(X_train, y_train)
        y_pred = model_instance.predict(X_test)

        # compute the confusion matrix
        cm = confusion_matrix(y_test, y_pred)

        # calculate precision, recall (sensitivity), f1-score
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # calculate specificity
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn+fp)

        # calculate MCC
        mcc = matthews_corrcoef(y_test, y_pred)

        accuracy = (tp + tn) / (tp + tn + fp + fn)

        # Set user attributes
        trial.set_user_attr("precision", precision)
        trial.set_user_attr("recall", recall)
        trial.set_user_attr("f1", f1)
        trial.set_user_attr("specificity", specificity)
        trial.set_user_attr("mcc", mcc)

        return accuracy

    study = optuna.create_study(direction='maximize')
    study.optimize(optuna_objective, n_trials=trials)

    temp_df = pd.DataFrame({
        'feature_type': feature_type, 
        'model': name, 
        'with_hypertuning': True,
        'best_params': [str(study.best_trial.params)],
        'accuracy': study.best_trial.value, 
        'sensitivity': study.best_trial.user_attrs['recall'], 
        'specificity': study.best_trial.user_attrs['specificity'], 
        'precision': study.best_trial.user_attrs['precision'], 
        'f1': study.best_trial.user_attrs['f1'], 
        'mcc': study.best_trial.user_attrs['mcc'],
        'index': f'{feature_type}_{name}_with_hypertuning'
        }, index=['index'])
    results_dataframe = pd.concat([results_dataframe, temp_df])
    return results_dataframe


# Define models
models = {
    'LogisticRegression': LogisticRegression(),
    'SVC': SVC(),
    'XGBClassifier': XGBClassifier(),
    'LGBMClassifier': LGBMClassifier()
}

models_ = {
    'LogisticRegression': LogisticRegression,
    'SVC': SVC,
    'XGBClassifier': XGBClassifier,
    'LGBMClassifier': LGBMClassifier
}

# Define objectives for hyperparameters tuning
objectives = {
    'LogisticRegression': lambda trial: {
        'C': trial.suggest_float('C', 1e-2, 1e-1),
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2']),
        'solver': trial.suggest_categorical('solver', ['liblinear', 'saga']),
        'max_iter': trial.suggest_int('max_iter', 100, 1000)
    },
    'SVC': lambda trial: {
        'C': trial.suggest_float('svc_c', 1e-2, 1e2),
        'gamma': trial.suggest_float('svc_gamma', 1e-2, 1e2),
    },
    'XGBClassifier': lambda trial: {
        'learning_rate': trial.suggest_float("learning_rate", 1e-2, 0.3),
        'max_depth': trial.suggest_int("max_depth", 2, 6),
        'n_estimators': trial.suggest_int("n_estimators", 100, 1000)
    },
    'LGBMClassifier': lambda trial: {
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'max_depth': trial.suggest_int('max_depth', 2, 50),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000)
    }
}

# Without Feature Selection

In [228]:
# empty dataframe to store results with the columns feature_type, model, with_hypertuning, accuracy, sensitivity, specificity, precision, f1, mcc
results = pd.DataFrame(columns=['feature_type', 'model', 'with_hypertuning', 'best_params', 'accuracy', 'sensitivity', 'specificity', 'precision', 'f1', 'mcc', 'index'])
feature_types = ['AAC', 'APAAC', 'CTD', 'DPC', 'PAAC']
for feature_type in feature_types:

    # Load the training dataset
    data = pd.read_csv(f'{data_dir}/TR_{feature_type}.csv')

    # Separate features and target
    X = data.drop(columns=['label', 'id'], axis=1)
    y = data['label']

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Evaluate models without hyperparameters tuning
    for name, model in models.items():
        print(f"Evaluating {feature_type} {name}")
        results = evaluate_model(name, model, X_train, y_train, X_test, y_test, results, feature_type)
        print(results)

    # Optimize hyperparameters
    for name, model in models_.items():
        objective = objectives.get(name)
        if objective is not None:
            print(f"Optimizing {feature_type} {name}")
            results = optimize_hyperparameters(name, model, objective, trials=100, results_dataframe=results, feature_type=feature_type, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)
            print(results)

results.to_csv('results_v2.csv', index=False)

Evaluating AAC LogisticRegression
      feature_type               model with_hypertuning best_params  accuracy   
index          AAC  LogisticRegression            False        None  0.875077  \

       sensitivity  specificity  precision        f1       mcc   
index     0.791837     0.901515   0.881818  0.834409  0.699324  \

                                       index  
index  AAC_LogisticRegression_no_hypertuning  
Evaluating AAC SVC
      feature_type               model with_hypertuning best_params  accuracy   
index          AAC  LogisticRegression            False        None  0.875077  \
index          AAC                 SVC            False        None  0.897209   

       sensitivity  specificity  precision        f1       mcc   
index     0.791837     0.901515   0.881818  0.834409  0.699324  \
index     0.836735     0.931818   0.919283  0.876068  0.773969   

                                       index  
index  AAC_LogisticRegression_no_hypertuning  
index               

[I 2023-05-14 10:09:55,554] A new study created in memory with name: no-name-daae56c1-c117-41a1-9c4a-b5bbc95a14d6
[I 2023-05-14 10:09:55,580] Trial 0 finished with value: 0.8428290766208252 and parameters: {'C': 0.024676775179329563, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 782}. Best is trial 0 with value: 0.8428290766208252.
[I 2023-05-14 10:09:55,600] Trial 1 finished with value: 0.8565815324165029 and parameters: {'C': 0.09094766750588791, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 431}. Best is trial 1 with value: 0.8565815324165029.
[I 2023-05-14 10:09:55,620] Trial 2 finished with value: 0.8546168958742633 and parameters: {'C': 0.06823992043723134, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 310}. Best is trial 1 with value: 0.8565815324165029.
[I 2023-05-14 10:09:55,639] Trial 3 finished with value: 0.8526522593320236 and parameters: {'C': 0.041434628004611083, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 950}. Best is trial 1 with value: 0.85

      feature_type               model with_hypertuning best_params  accuracy   
index          AAC  LogisticRegression            False        None  0.875077  \
index          AAC                 SVC            False        None  0.897209   
index          AAC       XGBClassifier            False        None  0.890297   
index          AAC      LGBMClassifier            False        None  0.883399   

       sensitivity  specificity  precision        f1       mcc   
index     0.791837     0.901515   0.881818  0.834409  0.699324  \
index     0.836735     0.931818   0.919283  0.876068  0.773969   
index     0.828571     0.928030   0.914414  0.869379  0.762316   
index     0.840816     0.909091   0.895652  0.867368  0.752881   

                                       index  
index  AAC_LogisticRegression_no_hypertuning  
index                 AAC_SVC_no_hypertuning  
index       AAC_XGBClassifier_no_hypertuning  
index      AAC_LGBMClassifier_no_hypertuning  
Optimizing AAC LogisticRegre

[I 2023-05-14 10:09:55,778] Trial 8 finished with value: 0.8546168958742633 and parameters: {'C': 0.09673575707935296, 'penalty': 'l2', 'solver': 'saga', 'max_iter': 178}. Best is trial 1 with value: 0.8565815324165029.
[I 2023-05-14 10:09:55,820] Trial 9 finished with value: 0.8506876227897839 and parameters: {'C': 0.03917073268527012, 'penalty': 'l2', 'solver': 'saga', 'max_iter': 153}. Best is trial 1 with value: 0.8565815324165029.
[I 2023-05-14 10:09:55,863] Trial 10 finished with value: 0.8526522593320236 and parameters: {'C': 0.09505930844922625, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 560}. Best is trial 1 with value: 0.8565815324165029.
[I 2023-05-14 10:09:55,908] Trial 11 finished with value: 0.8546168958742633 and parameters: {'C': 0.07594439674443194, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 354}. Best is trial 1 with value: 0.8565815324165029.
[I 2023-05-14 10:09:55,950] Trial 12 finished with value: 0.8526522593320236 and parameters: {'C': 0.0795446

      feature_type               model with_hypertuning   
index          AAC  LogisticRegression            False  \
index          AAC                 SVC            False   
index          AAC       XGBClassifier            False   
index          AAC      LGBMClassifier            False   
index          AAC  LogisticRegression             True   

                                             best_params  accuracy   
index                                               None  0.875077  \
index                                               None  0.897209   
index                                               None  0.890297   
index                                               None  0.883399   
index  {'C': 0.03277840391147581, 'penalty': 'l1', 's...  0.858546   

       sensitivity  specificity  precision        f1       mcc   
index     0.791837     0.901515   0.881818  0.834409  0.699324  \
index     0.836735     0.931818   0.919283  0.876068  0.773969   
index     0.828571     0.9

...

      feature_type               model with_hypertuning   
index          AAC  LogisticRegression            False  \
index          AAC                 SVC            False   
index          AAC       XGBClassifier            False   
index          AAC      LGBMClassifier            False   
index          AAC  LogisticRegression             True   
index          AAC                 SVC             True   
index          AAC       XGBClassifier             True   
index          AAC      LGBMClassifier             True   
index        APAAC  LogisticRegression            False   
index        APAAC                 SVC            False   
index        APAAC       XGBClassifier            False   
index        APAAC      LGBMClassifier            False   
index        APAAC  LogisticRegression             True   
index        APAAC                 SVC             True   
index        APAAC       XGBClassifier             True   
index        APAAC      LGBMClassifier             True 

[I 2023-05-14 11:18:15,177] Trial 0 finished with value: 0.8972332015810277 and parameters: {'learning_rate': 0.2619850011443755, 'max_depth': 2, 'n_estimators': 587}. Best is trial 0 with value: 0.8972332015810277.
[I 2023-05-14 11:18:16,030] Trial 1 finished with value: 0.8913043478260869 and parameters: {'learning_rate': 0.21638737361637084, 'max_depth': 2, 'n_estimators': 448}. Best is trial 0 with value: 0.8972332015810277.
[I 2023-05-14 11:18:17,753] Trial 2 finished with value: 0.8992094861660079 and parameters: {'learning_rate': 0.09799562429942851, 'max_depth': 3, 'n_estimators': 535}. Best is trial 2 with value: 0.8992094861660079.
[I 2023-05-14 11:18:19,494] Trial 3 finished with value: 0.8913043478260869 and parameters: {'learning_rate': 0.10373571257505594, 'max_depth': 5, 'n_estimators': 422}. Best is trial 2 with value: 0.8992094861660079.
[I 2023-05-14 11:18:20,306] Trial 4 finished with value: 0.8873517786561265 and parameters: {'learning_rate': 0.27986375169709904, 'm

      feature_type               model with_hypertuning   
index          AAC  LogisticRegression            False  \
index          AAC                 SVC            False   
index          AAC       XGBClassifier            False   
index          AAC      LGBMClassifier            False   
index          AAC  LogisticRegression             True   
index          AAC                 SVC             True   
index          AAC       XGBClassifier             True   
index          AAC      LGBMClassifier             True   
index        APAAC  LogisticRegression            False   
index        APAAC                 SVC            False   
index        APAAC       XGBClassifier            False   
index        APAAC      LGBMClassifier            False   
index        APAAC  LogisticRegression             True   
index        APAAC                 SVC             True   
index        APAAC       XGBClassifier             True   
index        APAAC      LGBMClassifier             True 

[I 2023-05-14 11:23:23,453] Trial 0 finished with value: 0.9071146245059288 and parameters: {'num_leaves': 151, 'max_depth': 16, 'learning_rate': 0.23393208125638126, 'n_estimators': 1876}. Best is trial 0 with value: 0.9071146245059288.
[I 2023-05-14 11:23:24,660] Trial 1 finished with value: 0.8952569169960475 and parameters: {'num_leaves': 19, 'max_depth': 13, 'learning_rate': 0.2325634678118051, 'n_estimators': 264}. Best is trial 0 with value: 0.9071146245059288.
[I 2023-05-14 11:23:25,500] Trial 2 finished with value: 0.8873517786561265 and parameters: {'num_leaves': 183, 'max_depth': 34, 'learning_rate': 0.29455523636689307, 'n_estimators': 242}. Best is trial 0 with value: 0.9071146245059288.
[I 2023-05-14 11:23:27,917] Trial 3 finished with value: 0.8932806324110671 and parameters: {'num_leaves': 186, 'max_depth': 48, 'learning_rate': 0.12323717380960289, 'n_estimators': 957}. Best is trial 0 with value: 0.9071146245059288.
[I 2023-05-14 11:23:31,246] Trial 4 finished with val

      feature_type               model with_hypertuning   
index          AAC  LogisticRegression            False  \
index          AAC                 SVC            False   
index          AAC       XGBClassifier            False   
index          AAC      LGBMClassifier            False   
index          AAC  LogisticRegression             True   
index          AAC                 SVC             True   
index          AAC       XGBClassifier             True   
index          AAC      LGBMClassifier             True   
index        APAAC  LogisticRegression            False   
index        APAAC                 SVC            False   
index        APAAC       XGBClassifier            False   
index        APAAC      LGBMClassifier            False   
index        APAAC  LogisticRegression             True   
index        APAAC                 SVC             True   
index        APAAC       XGBClassifier             True   
index        APAAC      LGBMClassifier             True 

# With Feature Selection

In [230]:
# empty dataframe to store results with the columns feature_type, model, with_hypertuning, accuracy, sensitivity, specificity, precision, f1, mcc
results = pd.DataFrame(columns=['feature_type', 'model', 'with_hypertuning', 'best_params', 'accuracy', 'sensitivity', 'specificity', 'precision', 'f1', 'mcc', 'index'])
feature_types = ['selected_features_all_best20', 'selected_features_all_best30', 'selected_features_all_best50', 'selected_features_all_best100']
for feature_type in feature_types:

    # Load the training dataset
    data = pd.read_csv(f'{feature_engineered_data_dir}/TR_{feature_type}.csv')

    # Separate features and target
    X = data.drop(columns=['label', 'id'], axis=1)
    y = data['label']

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Evaluate models without hyperparameters tuning
    for name, model in models.items():
        print(f"Evaluating {feature_type} {name}")
        results = evaluate_model(name, model, X_train, y_train, X_test, y_test, results, feature_type)
        print(results)

    # Optimize hyperparameters
    for name, model in models_.items():
        objective = objectives.get(name)
        if objective is not None:
            print(f"Optimizing {feature_type} {name}")
            results = optimize_hyperparameters(name, model, objective, trials=100, results_dataframe=results, feature_type=feature_type, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)
            print(results)

results.to_csv(f'{feature_engineered_data_dir}/results_20&30&50&100.csv', index=False)

Evaluating selected_features_all_best20 LogisticRegression
                       feature_type               model with_hypertuning   
index  selected_features_all_best20  LogisticRegression            False  \

      best_params  accuracy  sensitivity  specificity  precision        f1   
index        None   0.86163     0.832653     0.850575   0.839506  0.836066  \

           mcc                                              index  
index  0.68342  selected_features_all_best20_LogisticRegressio...  
Evaluating selected_features_all_best20 SVC
                       feature_type               model with_hypertuning   
index  selected_features_all_best20  LogisticRegression            False  \
index  selected_features_all_best20                 SVC            False   

      best_params  accuracy  sensitivity  specificity  precision        f1   
index        None  0.861630     0.832653     0.850575   0.839506  0.836066  \
index        None  0.884356     0.816327     0.915709   0.900901  

[I 2023-05-14 11:32:32,517] A new study created in memory with name: no-name-3df58edf-44c6-4cf2-9751-20d6cafa617a
[I 2023-05-14 11:32:32,628] Trial 0 finished with value: 0.857707509881423 and parameters: {'C': 0.07403839557995674, 'penalty': 'l2', 'solver': 'saga', 'max_iter': 496}. Best is trial 0 with value: 0.857707509881423.
[I 2023-05-14 11:32:32,657] Trial 1 finished with value: 0.8537549407114624 and parameters: {'C': 0.0810939833636358, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 522}. Best is trial 0 with value: 0.857707509881423.
[I 2023-05-14 11:32:32,684] Trial 2 finished with value: 0.857707509881423 and parameters: {'C': 0.08162923941197098, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 802}. Best is trial 0 with value: 0.857707509881423.


                       feature_type               model with_hypertuning   
index  selected_features_all_best20  LogisticRegression            False  \
index  selected_features_all_best20                 SVC            False   
index  selected_features_all_best20       XGBClassifier            False   
index  selected_features_all_best20      LGBMClassifier            False   

      best_params  accuracy  sensitivity  specificity  precision        f1   
index        None  0.861630     0.832653     0.850575   0.839506  0.836066  \
index        None  0.884356     0.816327     0.915709   0.900901  0.856531   
index        None  0.865564     0.840816     0.908046   0.895652  0.867368   
index        None  0.870999     0.832653     0.919540   0.906667  0.868085   

            mcc                                              index  
index  0.683420  selected_features_all_best20_LogisticRegressio...  
index  0.737224    selected_features_all_best20_SVC_no_hypertuning  
index  0.751600  sele

[I 2023-05-14 11:32:32,872] Trial 3 finished with value: 0.8596837944664032 and parameters: {'C': 0.06441953223187927, 'penalty': 'l1', 'solver': 'saga', 'max_iter': 574}. Best is trial 3 with value: 0.8596837944664032.
[I 2023-05-14 11:32:32,897] Trial 4 finished with value: 0.857707509881423 and parameters: {'C': 0.040412568834002696, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 939}. Best is trial 3 with value: 0.8596837944664032.
[I 2023-05-14 11:32:32,923] Trial 5 finished with value: 0.857707509881423 and parameters: {'C': 0.026477898873890016, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 184}. Best is trial 3 with value: 0.8596837944664032.
[I 2023-05-14 11:32:33,059] Trial 6 finished with value: 0.8616600790513834 and parameters: {'C': 0.03342895231698814, 'penalty': 'l1', 'solver': 'saga', 'max_iter': 773}. Best is trial 6 with value: 0.8616600790513834.
[I 2023-05-14 11:32:33,079] Trial 7 finished with value: 0.8537549407114624 and parameters: {'C': 0.0473230574

                       feature_type               model with_hypertuning   
index  selected_features_all_best20  LogisticRegression            False  \
index  selected_features_all_best20                 SVC            False   
index  selected_features_all_best20       XGBClassifier            False   
index  selected_features_all_best20      LGBMClassifier            False   
index  selected_features_all_best20  LogisticRegression             True   

                                             best_params  accuracy   
index                                               None  0.861630  \
index                                               None  0.884356   
index                                               None  0.865564   
index                                               None  0.870999   
index  {'C': 0.05772645471274076, 'penalty': 'l1', 's...  0.863636   

       sensitivity  specificity  precision        f1       mcc   
index     0.832653     0.850575   0.839506  0.836066  0.

...

                        feature_type               model with_hypertuning   
index   selected_features_all_best20  LogisticRegression            False  \
index   selected_features_all_best20                 SVC            False   
index   selected_features_all_best20       XGBClassifier            False   
index   selected_features_all_best20      LGBMClassifier            False   
index   selected_features_all_best20  LogisticRegression             True   
index   selected_features_all_best20                 SVC             True   
index   selected_features_all_best20       XGBClassifier             True   
index   selected_features_all_best20      LGBMClassifier             True   
index   selected_features_all_best30  LogisticRegression            False   
index   selected_features_all_best30                 SVC            False   
index   selected_features_all_best30       XGBClassifier            False   
index   selected_features_all_best30      LGBMClassifier            False   

[I 2023-05-14 11:59:21,611] Trial 0 finished with value: 0.8893280632411067 and parameters: {'learning_rate': 0.21246312417174512, 'max_depth': 2, 'n_estimators': 616}. Best is trial 0 with value: 0.8893280632411067.
[I 2023-05-14 11:59:25,717] Trial 1 finished with value: 0.8972332015810277 and parameters: {'learning_rate': 0.05172888818693755, 'max_depth': 5, 'n_estimators': 403}. Best is trial 1 with value: 0.8972332015810277.
[I 2023-05-14 11:59:30,699] Trial 2 finished with value: 0.8932806324110671 and parameters: {'learning_rate': 0.05498622277393403, 'max_depth': 4, 'n_estimators': 496}. Best is trial 1 with value: 0.8972332015810277.
[I 2023-05-14 11:59:33,145] Trial 3 finished with value: 0.8873517786561265 and parameters: {'learning_rate': 0.2611868850156039, 'max_depth': 4, 'n_estimators': 354}. Best is trial 1 with value: 0.8972332015810277.
[I 2023-05-14 11:59:38,608] Trial 4 finished with value: 0.8932806324110671 and parameters: {'learning_rate': 0.1243897891289126, 'ma

                        feature_type               model with_hypertuning   
index   selected_features_all_best20  LogisticRegression            False  \
index   selected_features_all_best20                 SVC            False   
index   selected_features_all_best20       XGBClassifier            False   
index   selected_features_all_best20      LGBMClassifier            False   
index   selected_features_all_best20  LogisticRegression             True   
index   selected_features_all_best20                 SVC             True   
index   selected_features_all_best20       XGBClassifier             True   
index   selected_features_all_best20      LGBMClassifier             True   
index   selected_features_all_best30  LogisticRegression            False   
index   selected_features_all_best30                 SVC            False   
index   selected_features_all_best30       XGBClassifier            False   
index   selected_features_all_best30      LGBMClassifier            False   

[I 2023-05-14 12:06:48,046] Trial 0 finished with value: 0.8873517786561265 and parameters: {'num_leaves': 98, 'max_depth': 22, 'learning_rate': 0.26204601918854353, 'n_estimators': 444}. Best is trial 0 with value: 0.8873517786561265.
[I 2023-05-14 12:06:49,925] Trial 1 finished with value: 0.8913043478260869 and parameters: {'num_leaves': 37, 'max_depth': 32, 'learning_rate': 0.0747843128262703, 'n_estimators': 364}. Best is trial 1 with value: 0.8913043478260869.
[I 2023-05-14 12:06:53,805] Trial 2 finished with value: 0.8893280632411067 and parameters: {'num_leaves': 177, 'max_depth': 32, 'learning_rate': 0.14651735429553095, 'n_estimators': 1942}. Best is trial 1 with value: 0.8913043478260869.
[I 2023-05-14 12:06:56,951] Trial 3 finished with value: 0.8913043478260869 and parameters: {'num_leaves': 120, 'max_depth': 8, 'learning_rate': 0.09491243639916087, 'n_estimators': 1849}. Best is trial 1 with value: 0.8913043478260869.
[I 2023-05-14 12:06:59,431] Trial 4 finished with valu

                        feature_type               model with_hypertuning   
index   selected_features_all_best20  LogisticRegression            False  \
index   selected_features_all_best20                 SVC            False   
index   selected_features_all_best20       XGBClassifier            False   
index   selected_features_all_best20      LGBMClassifier            False   
index   selected_features_all_best20  LogisticRegression             True   
index   selected_features_all_best20                 SVC             True   
index   selected_features_all_best20       XGBClassifier             True   
index   selected_features_all_best20      LGBMClassifier             True   
index   selected_features_all_best30  LogisticRegression            False   
index   selected_features_all_best30                 SVC            False   
index   selected_features_all_best30       XGBClassifier            False   
index   selected_features_all_best30      LGBMClassifier            False   

# Best Model with Full Training Dataset

In [248]:
# Load the results
results_without_selected_features = pd.read_csv('results_v2.csv')
results_with_selected_features = pd.read_csv(f'{feature_engineered_data_dir}/results_20&30&50&100.csv')

feature_types = ['AAC', 'APAAC', 'CTD', 'DPC', 'PAAC']
selected_feature_types = ['selected_features_all_best20', 'selected_features_all_best30', 'selected_features_all_best50', 'selected_features_all_best100']

# Combine the feature types
feature_types.extend(selected_feature_types)

test_results = []

# iterate through each row of results
for feature_type in feature_types:

    # Check if the feature type is selected features
    if 'selected_features' in feature_type:
        # Load the training dataset
        train_data = pd.read_csv(f'{feature_engineered_data_dir}/TR_{feature_type}.csv')
        test_data = pd.read_csv(f'{feature_engineered_data_dir}/TS_{feature_type}.csv')
        results = results_with_selected_features
    else:
        # Load the training dataset
        train_data = pd.read_csv(f'{data_dir}/TR_{feature_type}.csv')
        test_data = pd.read_csv(f'{data_dir}/TS_{feature_type}.csv')
        results = results_without_selected_features

    # Separate features and target
    X_train = train_data.drop(columns=['label', 'id'], axis=1)
    y_train = train_data['label']

    X_test = test_data.drop(columns=['label', 'id'], axis=1)
    y_test = test_data['label']

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # iterate through each model
    for name, model in models.items():
        # get the row of the model

        
        rows = results[(results['feature_type'] == feature_type) & (results['model'] == name)]

        # iterate through each row
        for index, row in rows.iterrows():

            # check whether the model has hyperparameters
            if row['with_hypertuning'] == True:
                hyperparameters = ast.literal_eval(row['best_params'])
                # check the model is SVC
                if row['model'] == 'SVC':
                    hyperparameters = {k[4:]: v for k, v in hyperparameters.items()}
                    # make key 'c' to 'C'
                    hyperparameters['C'] = hyperparameters.pop('c')
                # set best hyperparameters
                model.set_params(**hyperparameters)

            # fit model
            model.fit(X_train, y_train)

            # predict
            y_pred = model.predict(X_test)

            # evaluate using accuracy, sensitivity, specificity, precision, f1, mcc
            accuracy = accuracy_score(y_test, y_pred)
            sensitivity = recall_score(y_test, y_pred)
            specificity = recall_score(y_test, y_pred, pos_label=0)
            precision = precision_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            mcc = matthews_corrcoef(y_test, y_pred)

            # append to test_results
            test_results.append({'feature_type': feature_type, 'model': name, 'with_hypertuning': row['with_hypertuning'], 'best_params': row['best_params'], 'accuracy': accuracy, 'sensitivity': sensitivity, 'specificity': specificity, 'precision': precision, 'f1': f1, 'mcc': mcc, 'index': row['index']})
    print(f'Feature Type: {feature_type} done!')

test_results = pd.DataFrame(test_results)
test_results.to_csv('test_results.csv', index=False)

Feature Type: AAC done!
Feature Type: APAAC done!


/home/darshana/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Feature Type: CTD done!


/home/darshana/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Feature Type: DPC done!
Feature Type: PAAC done!
Feature Type: selected_features_all_best20 done!
Feature Type: selected_features_all_best30 done!


/home/darshana/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Feature Type: selected_features_all_best50 done!


/home/darshana/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/darshana/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Feature Type: selected_features_all_best100 done!


# Model Ensembling

# Model Ensembling with Selected Features